In [1]:
!pip install pulp

In [ ]:
from openpyxl import load_workbook

# Load Excel file
input_file = "input.xlsx" #Change name for each execution
wb = load_workbook(input_file, data_only=True)
ws = wb["Sheet1"]

def get_matrix(cell_range):
    return [[(cell.value if cell.value is not None else 0) for cell in row] for row in ws[cell_range]]  #transform "None" into 0

# Real Data
maxruns = ws["AK3"].value
I = get_matrix("AI3:AI14")
I = [(x[0]) for x in I] #transform into list of values
B = get_matrix("W3:AH14")
S  = get_matrix("I3:T14")
T = get_matrix("T24:T4463")
T = [(x[0]) for x in T]
F = get_matrix("AJ3:AJ14")
F = [(x[0]) for x in F]
Hmin = get_matrix("AA48:CV59")
Hmax = get_matrix("AA63:CV74")
LineStart = get_matrix("AA77:AA88")
LineStart = [(x[0]) for x in LineStart]
LineFinish = get_matrix("AB77:AB88")
LineFinish = [(x[0]) for x in LineFinish]

# Dummy Data
Dev = get_matrix("O24:O4463")
Dev = [(x[0]) for x in Dev]
N = get_matrix("E24:E28")
N = [(x[0]) for x in N]
W = get_matrix("J24:J743")
W = [(x[0]) for x in W]

def get_matrix_none(cell_range):
    return [[(cell.value) for cell in row] for row in ws[cell_range]]  #keep "None" for later calculations

# Bunching
TB = get_matrix_none("T24:T4463")

In [ ]:
import pulp
from openpyxl import Workbook

# Problem
model = pulp.LpProblem('Timetable_Synchronization', pulp.LpMaximize)


# Input Data
lines = 12
syncnodesno = 5
L1 = range(0, lines)   #0 a lines-1 em vez de 1 a lines
Runs = range(0, maxruns)
SynchNodes = range(0, syncnodesno)
WArray = [[[0 for s in SynchNodes] for q in L1] for p in L1]
DevArray = [[[0 for p in Runs] for s in SynchNodes] for i in L1]  # i = line, p = run, s = node
TArray = [[[0 for p in Runs] for s in SynchNodes] for i in L1]
TBArray = [[[0 for p in Runs] for s in SynchNodes] for i in L1]

weights = dict() #for faster computation
for i in L1:
    for j in range(I[i]):
        target = S[i][B[i][j] - 1] - 1
        weights[(i, j)] = N[target]

for i in L1:
  for j in L1:
    for s in SynchNodes:
        WArray[i][j][s] = W[s + syncnodesno * j + lines * syncnodesno * i]

for i in L1:
  for s in SynchNodes:
    for p in Runs:
        DevArray[i][s][p] = Dev[p + maxruns * s + syncnodesno * maxruns * i]
        TArray[i][s][p] = T[p + maxruns * s + syncnodesno * maxruns * i]
        TBArray[i][s][p] = TB[p + maxruns * s + syncnodesno * maxruns * i]


# Decision Variables
Y = pulp.LpVariable.dicts('Y', [(i, j, s, p, q) for i in L1 for j in L1 for s in SynchNodes for p in Runs for q in Runs], cat=pulp.LpBinary)
X = pulp.LpVariable.dicts('X', [(i, p) for i in L1 for p in Runs], cat=pulp.LpInteger)


# Objective Function
model += sum(weights[(i, j)] * Y[(i, B[i][j] - 1, S[i][B[i][j] - 1]- 1, p, q)] \
            for i in L1 for j in range(0, I[i]) for p in range(0, F[i]) for q in range(0, F[B[i][j] - 1]))


# Constraints

# Earliest and latest departures
for i in L1:
  model += X[(i, 0)] >= LineStart[i]

for i in L1:
  for p in range(0, F[i]):
      model += X[(i, p)] <= LineFinish[i]  + 100   #preciso aumentar a margem deste valor (folga tempo limite)

# Headways
for i in L1:
  for p in range(0, F[i]-1):
    model += Hmin[i][p] <= X[(i, p+1)] - X[(i, p)]
    model += X[(i, p+1)] - X[(i, p)] <= Hmax[i][p]

# Synchronization
for i in L1:
  for j in range(0, I[i]):
    for p in range(0, F[i]):
      for q in range(0, F[B[i][j] - 1]):
        model += (X[(B[i][j] - 1, q)] + TArray[B[i][j] - 1][S[i][B[i][j] - 1] - 1][q] + DevArray[B[i][j] - 1][S[i][B[i][j] - 1] - 1][q]) \
                - (X[(i, p)] + TArray[i][S[i][B[i][j] - 1] - 1][p] + DevArray[i][S[i][B[i][j] - 1] - 1][p] ) \
                >=  WArray[i][B[i][j] - 1][S[i][B[i][j] - 1] - 1] - 100000 * (1 - Y[(i, B[i][j] - 1, S[i][B[i][j] - 1] - 1, p, q)])
        model += (X[(B[i][j] - 1, q)] + TArray[B[i][j] - 1][S[i][B[i][j] - 1] - 1][q] + DevArray[B[i][j] - 1][S[i][B[i][j] - 1] - 1][q]) \
                - (X[(i, p)] + TArray[i][S[i][B[i][j] - 1] - 1][p] + DevArray[i][S[i][B[i][j] - 1] - 1][p] ) \
                <=  (WArray[i][B[i][j] - 1][S[i][B[i][j] - 1] - 1] + 15)  +  100000 * (1 - Y[(i, B[i][j] - 1, S[i][B[i][j] - 1] - 1, p, q)])

# Force Y=0 in skipped nodes
for q in Runs:
  model += Y[(3-1, 6-1, S[3-1][B[3-1][6-1] - 1] - 1, 2-1, q)] == 0 #Y3,6,2,2,all
  model += Y[(5-1, 4-1, S[5-1][B[5-1][4-1] - 1] - 1, 8-1, q)] == 0 #Y5,4,2,8,all
  model += Y[(5-1, 4-1, S[5-1][B[5-1][4-1] - 1] - 1, 9-1, q)] == 0 #Y5,4,2,9,all
  model += Y[(10-1, 11-1, S[10-1][B[10-1][11-1] - 1] - 1, 54-1, q)] == 0 #Y10,11,5,54,all
  model += Y[(10-1, 12-1, S[10-1][B[10-1][12-1] - 1] - 1, 54-1, q)] == 0 #Y10,12,5,54,all
  model += Y[(12-1, 1-1, S[12-1][B[12-1][1-1] - 1] - 1, 20-1, q)] == 0 #Y12,1,3,20,all
  model += Y[(12-1, 2-1, S[12-1][B[12-1][2-1] - 1] - 1, 20-1, q)] == 0 #Y12,2,3,20,all
  model += Y[(12-1, 3-1, S[12-1][B[12-1][3-1] - 1] - 1, 20-1, q)] == 0 #Y12,3,3,20,all
  model += Y[(12-1, 4-1, S[12-1][B[12-1][4-1] - 1] - 1, 20-1, q)] == 0 #Y12,4,3,20,all
  model += Y[(12-1, 5-1, S[12-1][B[12-1][5-1] - 1] - 1, 20-1, q)] == 0 #Y12,5,3,20,all
  model += Y[(12-1, 6-1, S[12-1][B[12-1][6-1] - 1] - 1, 20-1, q)] == 0 #Y12,6,3,20,all


# Solver
solver = pulp.PULP_CBC_CMD(
    threads=8,
    timeLimit=90,
)

model.solve(solver)

In [ ]:
# Ouput in Excel
import re

wb = Workbook()

  ### Timetables
ws1 = wb.active
ws1.title = "Timetable"

ws1.append(["Line", "Stop", "Kms"]+ [f"Run {p}" for p in Runs])

for i in L1:
  row = [i, "Start",""]
  for p in range(0, F[i]):
      row.append(X[(i, p)].varValue)
  ws1.append(row)
  for s in SynchNodes:
    row = [i,s,""]
    if TBArray[i][s][2] != [None]: #2 because travel times always exist in run 3 of every line
      for p in range(0, F[i]):
        value = X[(i, p)].varValue + TArray[i][s][p]
        row.append(value)
      ws1.append(row)
  row = [i, "End"]
  ws1.append(row)

  ### Synchronizations
ws2 = wb.create_sheet(title="Synchronizations")

ws2.append(['Y', 'i', 'j', 's', 'p', 'q', 'X[(i,p)]', 'T[i][s][p]', 'D[i][s][p]', 'X[(j,q)]', 'T[j][s][q]', 'D[j][s][q]', 'WArray[i][j][s]'])

for i in L1:
    for j in range(0, I[i]):
         for p in range(0, F[i]):
            for q in range(0, F[B[i][j] - 1]):
                 if round(Y[(i, B[i][j] - 1, S[i][B[i][j] - 1]- 1, p, q)].varValue) == 1:
                     ws2.append([Y[(i, B[i][j] - 1, S[i][B[i][j] - 1]- 1, p, q)].varValue, i, B[i][j] - 1, S[i][B[i][j] - 1]- 1, p, q, X[(i,p)].varValue, TArray[i][S[i][B[i][j] - 1]- 1][p], DevArray[i][S[i][B[i][j] - 1]- 1][p], X[(B[i][j] - 1,q)].varValue, TArray[B[i][j] - 1][S[i][B[i][j] - 1]- 1][q], DevArray[B[i][j] - 1][S[i][B[i][j] - 1]- 1][q], WArray[i][B[i][j] - 1][S[i][B[i][j] - 1]- 1]])

  ### Bunching
ws3 = wb.create_sheet(title="Bunching")

ws3.append(['i', 'j', 's', 'p', 'q', 'X[(i,p)]', 'T[i][s][p]', 'D[i][s][p]', 'X[(j,q)]', 'T[j][s][q]', 'D[j][s][q]'])
bus_pairs = [{6,9}, {7,9}] #bus pairs where bunching is possible (always Ermesinde)
train_pairs_lousado = [{3,5}, {2,4}] #train pairs where bunching is problematic in Lousado due to line constraints
train_pairs_ermesinde = [{1,3}, {1,5}, {3,5}, {0,2}, {0,4}, {2,4}] #train pairs where bunching is problematic in Ermesinde due to line constraints

for i in L1:
    for j in L1:
      if {i, j} in bus_pairs:
            s = 0  #Ermesinde
            for p in Runs:
                for q in Runs:
                   if TBArray[i][s][p] != [None] and TBArray[j][s][q] != [None]:
                     if abs((X[(i, p)].varValue + TArray[i][s][p] +  DevArray[i][s][p]) - (X[(j, q)].varValue + TArray[j][s][q] + DevArray[j][s][q])) <= 1 : #arriving at the same stop under 1 minute
                        ws3.append([i, j, s, p, q, X[(i, p)].varValue, TArray[i][s][p], DevArray[i][s][p], X[(j, q)].varValue, TArray[j][s][q], DevArray[j][s][q]])
      if {i, j} in train_pairs_lousado:
            s = 1  #Lousado
            for p in Runs:
                for q in Runs:
                   if TBArray[i][s][p] != [None] and TBArray[j][s][q] != [None]:
                      if abs((X[(i, p)].varValue + TArray[i][s][p] +  DevArray[i][s][p]) - (X[(j, q)].varValue + TArray[j][s][q] + DevArray[j][s][q])) <= 3 : #arriving at the same stop under 3 minutes
                        ws3.append([i, j, s, p, q, X[(i, p)].varValue, TArray[i][s][p], DevArray[i][s][p], X[(j, q)].varValue, TArray[j][s][q], DevArray[j][s][q]])
      if {i, j} in train_pairs_ermesinde:
            s = 0  #Ermesinde
            for p in Runs:
                for q in Runs:
                   if TBArray[i][s][p] != [None] and TBArray[j][s][q] != [None]:
                      if abs((X[(i, p)].varValue + TArray[i][s][p] +  DevArray[i][s][p]) - (X[(j, q)].varValue + TArray[j][s][q] + DevArray[j][s][q])) <= 3 : #arriving at the same stop under 3 minutes
                        ws3.append([i, j, s, p, q, X[(i, p)].varValue, TArray[i][s][p], DevArray[i][s][p], X[(j, q)].varValue, TArray[j][s][q], DevArray[j][s][q]])

# Extract the number (or identifier) from the input file name
match = re.search(r'Input_(\d+)\.xlsx', input_file)
if match:
    identifier = match.group(1)
    output_file = f"Output_{identifier}.xlsx"
else:
    output_file = "Output.xlsx"  # fallback if pattern not matched

wb.save(output_file)